In [ ]:
# Importamos las librerías necesarias
import requests # Para hacer solicitudes HTTP a la API
import json # Para trabajar con datos en formato JSON
from IPython.display import JSON # Para mostrar datos en formato JSON dentro de Jupyter
from keys import * # Importamos la clave API guardada en un archivo separado (keys.py)

In [ ]:
#be careful not to run too often as there is a maximum of 200 api calls per month

# Definimos la URL de la API para obtener vuelos que lleguen al aeropuerto JFK (KJFK) en un rango de tiempo específico
url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/KJFK/2025-09-01T08:00/2025-09-01T20:00"

# Definimos los parámetros que queremos incluir en la consulta (querystring)
querystring = {
    "withLeg":"true", #Incluye información de segmentos de vuelo
    "direction":"Arrival", #Solo llegadas
    "withCancelled":"true", #Vuelos cancelados
    "withCodeshared":"true",#Cuelos compartidos con otras aerolineas
    "withCargo":"true", #Vuelos de carga
    "withPrivate":"true", #Vuelos privados
    "withLocation":"false" #No incluye ubicacion en tiempo real
    }

#Los encabezados necesarios para autenticar la solicitud API
headers = {
    'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
    'x-rapidapi-key': flights_key
    }

#Se hace la petición GET a la API con los parametros y encabezados
response = requests.request("GET", url, headers=headers, params=querystring)

In [ ]:
#Verificamos si la solicitud fue exitosa
if response.status_code == 200:
    new_data = response.json() #convertimos la respuesta de la API en JSON

#Intentamos abrir el archivo 'data.json' donde guardamos la información previa
    try:
        with open("data.json", "r") as json_file:
            existing_data = json.load(json_file) #Cargamos los datos existentes
    except (FileNotFoundError, json.decoder.JSONDecodeError):
        existing_data = [] # Si no existe el archivo o esta vacio iniciamos una lista vacia

#Agregamos la nueva informacion a los datos ya existentes
    existing_data.append(new_data)

#Guardamos todo nuevamente en 'data.json'
    with open("data.json", "w") as json_file:
        json.dump(existing_data, json_file, indent=4)
        print("Data appended to data.json file.") #Este mensaje confirma la creacion del archivo
else:
    print("Failed to retrieve data from the API. Status code:", response.status_code) #Si la respuesta falla se muestra el error

Data appended to data.json file.


In [ ]:
#Guardamos la respuesta completa de la API en una variable
flight_arrivals = response.json()

#Accedemos a algunos campos de la respuesta para ver su estructura
for flight in flight_arrivals:
    flight_arrivals['arrivals'][0]['arrival']['scheduledTime'] #Hora de llegada
    flight_arrivals['arrivals'][0]['number'] #Numero de vuelo
    flight_arrivals['arrivals'][0]['arrival']['terminal'] #Terminal de llegada
    flight_arrivals['arrivals'][0]['departure']['airport']['name'] #Aeropuerto de origen
    flight_arrivals['arrivals'][0]['aircraft']['model'] #Modelo de avión
    flight_arrivals['arrivals'][0]['airline']['name'] #Aerolinea

In [ ]:
#Importamos las librerias para datos tabulares
import pandas as pd
from numpy import nan

#Volvemos a guardar la respuesta en la variable
flight_arrivals = response.json()

#En nuestra funcion extraemos informacion de un vuelo para pasarlo a un DataFrame
def flight_extraction(flights):
    flights_df = pd.json_normalize({
        'Scheduled_arrival_time': flights['arrival']['scheduledTime'], #Hora de llegada
        'flight_number': flights['number'], #Numero de vuelo
        'from' : flights['departure']['airport']['name'], #Aeropuerto de origen
        'airline' : flights['airline']['name'], #Aerolinea
        'aircraft' : flights['aircraft']['model'] #Modelo de avión
    })
    return(flights_df)

#Aplicamos la funcion a cada vuelo de la lista 'arrivals'
#y concatenamos los resultados en un solo DataFrame
flight_arrivals = pd.concat(
    [flight_extraction(flight) for flight in flight_arrivals['arrivals']]
)

#Finalmente mostramos las primeras filas
flight_arrivals.head()

,flight_number,from,airline,aircraft,Scheduled_arrival_time.utc,Scheduled_arrival_time.local
0,B6 80,Reno,JetBlue,BCS3,2025-09-01 11:30Z,2025-09-01 07:30-04:00
0,DL 5351,Richmond,Delta Air Lines,Canadair CRJ 900,2025-09-01 12:34Z,2025-09-01 08:34-04:00
0,DL 5284,Buffalo,Delta Air Lines,Canadair CRJ 900,2025-09-01 12:35Z,2025-09-01 08:35-04:00
0,B6 1224,Los Angeles,JetBlue,Airbus A321 (Sharklets),2025-09-01 12:30Z,2025-09-01 08:30-04:00
0,EY 1,Abu Dhabi Island and Internal Islands City,Etihad,Airbus A350-1000,2025-09-01 13:00Z,2025-09-01 09:00-04:00
